In [1]:
import pandas as pd
import numpy as np
import selection
import weights
import btag
import root_pandas
import importlib
import jetmet
import test_weights
import object_selection

Welcome to JupyROOT 6.20/06


In [232]:
importlib.reload(selection)
importlib.reload(jetmet)
importlib.reload(test_weights)
importlib.reload(object_selection)
path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + "T_TuneZ2_s" + ".root"
df, jet, tau, cut_flow = selection.event_selection(path, isData = False, isTT = True, corrLevel = "centJER")

Processing: T_TuneZ2_s isData: False isTT: True corrLevel centJER
SF 1.0 1.0
(30, 10) (30, 4)
eff [[0.73355883 0.01414611 0.013459794 0.6844905] [0.013669968 0.0127615575 0.01442174 0.4758115 0.014091027 0.6604813] [0.015353122 0.6464147 0.75262946 0.73776287 0.019866053] ... [0.011759935 0.7298863 0.740727 0.014267276] [0.7298863 0.7629098 0.21641394 0.015026693] [0.740727 0.013364381 0.65340453 0.024175556]]


In [255]:
pd.DataFrame.from_dict(cut_flow, orient='index')

,0
preselected,1915
hlt,273
jet_requirement,185
tau_requirement,30
lep_veto,30
met,23
btag,22


In [2]:
data = ['Run2011A_MultiJet', 'Run2011B_MultiJet']
mc = ['T_TuneZ2_s', 'WJetsToLNu', 'DYJetsToLL', 'T_TuneZ2_tW', 'T_TuneZ2_t-channel',
       'Tbar_TuneZ2_s', 'Tbar_TuneZ2_tW', 'Tbar_TuneZ2_t-channel'] #, 'TTJets']

In [227]:
importlib.reload(selection)
importlib.reload(jetmet)
importlib.reload(test_weights)
importlib.reload(object_selection)
def event_selection():
    
    samples = {}
    cut_flow = []
    cut_flow_normed = {}
    
    for sample in mc + data:
        
        #!!!!!!! Careful with JER application before Tau?!
        
        if "TTJets" in sample: isTT = True
        else: isTT = False
            
        if "Run2011" in sample: isData = True
        else: isData = False
        
        path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + sample + ".root"
        
        if isData:
            for inv_btag, suffix in zip([False, True], ["", "_QCD"]):
                
                df, _, _, ev_count, w_counts = selection.event_selection(path, isData = isData, isTT = isTT, 
                                                                          invert_btag = inv_btag)
                samples[sample + suffix] = df
                ev_count["sample"] = sample + suffix
                cut_flow.append(ev_count)
                cut_flow_normed[sample + suffix] = w_counts
        else:
            df, _, _, ev_count, w_counts = selection.event_selection(path, isData = isData, isTT = isTT)
            samples[sample] = df
            ev_count["sample"] = sample
            cut_flow.append(ev_count)
            cut_flow_normed[sample] = w_counts

        #print( cut_flow )        
        
    return samples, cut_flow, cut_flow_normed

samples, cut_flow, cut_flow_normed = event_selection()

Processing: T_TuneZ2_s isData: False isTT: False corrLevel cent invert_btag False
(30, 9) (30, 4)
(95,) (95,)
Processing: WJetsToLNu isData: False isTT: False corrLevel cent invert_btag False
(252, 9) (252, 4)
(173,) (173,)
Processing: DYJetsToLL isData: False isTT: False corrLevel cent invert_btag False
(710, 9) (710, 4)
(368,) (368,)
Processing: T_TuneZ2_tW isData: False isTT: False corrLevel cent invert_btag False
(632, 9) (632, 4)
(1672,) (1672,)
Processing: T_TuneZ2_t-channel isData: False isTT: False corrLevel cent invert_btag False
(245, 9) (245, 4)
(642,) (642,)
Processing: Tbar_TuneZ2_s isData: False isTT: False corrLevel cent invert_btag False
(13, 9) (13, 4)
(28,) (28,)
Processing: Tbar_TuneZ2_tW isData: False isTT: False corrLevel cent invert_btag False
(713, 9) (713, 4)
(1933,) (1933,)
Processing: Tbar_TuneZ2_t-channel isData: False isTT: False corrLevel cent invert_btag False
(150, 9) (150, 4)
(402,) (402,)
Processing: Run2011A_MultiJet isData: True isTT: False corrLevel 

Warning in <TROOT::Append>: Replacing existing TH1: T_TuneZ2_s (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: WJetsToLNu (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: DYJetsToLL (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: T_TuneZ2_tW (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: T_TuneZ2_t-channel (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Tbar_TuneZ2_s (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Tbar_TuneZ2_tW (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Tbar_TuneZ2_t-channel (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Run2011A_MultiJet (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Run2011A_MultiJet (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Run2011B_MultiJet (Potentia

In [274]:
df = pd.DataFrame.from_dict(cut_flow)

In [280]:
df_data = df[df["tau_requirement_w"].isna() == True]
df_mc = df[df["tau_requirement_w"].isna() == False]

In [281]:
df_data = df_data.astype(str)

In [283]:
def format_yield(col, col_err):
    return '{0:.1f}'.format(col) + " $\pm$ " + '{0:.1f}'.format(col_err)

In [296]:
out_cols = []
for cut in ["tau_requirement_w", "lep_veto_w", "met_w", "btag_w"]:
    
    df_mc[cut[:-2]] = df_mc.apply(lambda row : format_yield(row[cut], row[cut + "_err"]), axis=1)  

/usr/local/bin/start_ipykernel.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [297]:
df_mc

,btag,btag_w,btag_w_err,hlt,jet_requirement,lep_veto,lep_veto_w,lep_veto_w_err,met,met_w,...,sample,tau_requirement,tau_requirement_w,tau_requirement_w_err,trigger,tau_requirement_w_out,lep_veto_w_out,met_w_out,btag_w_out,_w
0,0.7 $\pm$ 0.2,0.73,0.16,273,184,1.0 $\pm$ 0.2,1.02,0.19,0.8 $\pm$ 0.2,0.79,...,T_TuneZ2_s,1.0 $\pm$ 0.2,1.02,0.19,nan,1.0 $\pm$ 0.2,1.0 $\pm$ 0.2,0.8 $\pm$ 0.2,0.7 $\pm$ 0.2,0.7 $\pm$ 0.2
1,42.7 $\pm$ 6.9,42.72,6.86,4982,3163,256.3 $\pm$ 17.2,256.31,17.18,237.1 $\pm$ 16.6,237.10,...,WJetsToLNu,280.6 $\pm$ 18.0,280.63,18.00,nan,280.6 $\pm$ 18.0,256.3 $\pm$ 17.2,237.1 $\pm$ 16.6,42.7 $\pm$ 6.9,42.7 $\pm$ 6.9
2,19.2 $\pm$ 2.1,19.15,2.08,19961,13383,122.3 $\pm$ 5.3,122.26,5.33,96.7 $\pm$ 4.7,96.67,...,DYJetsToLL,158.9 $\pm$ 6.1,158.85,6.08,nan,158.9 $\pm$ 6.1,122.3 $\pm$ 5.3,96.7 $\pm$ 4.7,19.2 $\pm$ 2.1,19.2 $\pm$ 2.1
3,10.5 $\pm$ 0.6,10.51,0.55,5529,3764,16.1 $\pm$ 0.7,16.12,0.69,13.8 $\pm$ 0.6,13.78,...,T_TuneZ2_tW,18.0 $\pm$ 0.7,17.96,0.73,nan,18.0 $\pm$ 0.7,16.1 $\pm$ 0.7,13.8 $\pm$ 0.6,10.5 $\pm$ 0.6,10.5 $\pm$ 0.6
4,4.9 $\pm$ 0.4,4.88,0.42,3396,2007,8.1 $\pm$ 0.5,8.06,0.53,5.6 $\pm$ 0.4,5.65,...,T_TuneZ2_t-channel,8.2 $\pm$ 0.5,8.24,0.54,nan,8.2 $\pm$ 0.5,8.1 $\pm$ 0.5,5.6 $\pm$ 0.4,4.9 $\pm$ 0.4,4.9 $\pm$ 0.4
5,0.2 $\pm$ 0.1,0.19,0.07,158,101,0.4 $\pm$ 0.1,0.37,0.11,0.3 $\pm$ 0.1,0.28,...,Tbar_TuneZ2_s,0.4 $\pm$ 0.1,0.39,0.11,nan,0.4 $\pm$ 0.1,0.4 $\pm$ 0.1,0.3 $\pm$ 0.1,0.2 $\pm$ 0.1,0.2 $\pm$ 0.1
6,11.4 $\pm$ 0.6,11.40,0.55,6034,4142,17.1 $\pm$ 0.7,17.09,0.68,14.7 $\pm$ 0.6,14.74,...,Tbar_TuneZ2_tW,18.8 $\pm$ 0.7,18.79,0.72,nan,18.8 $\pm$ 0.7,17.1 $\pm$ 0.7,14.7 $\pm$ 0.6,11.4 $\pm$ 0.6,11.4 $\pm$ 0.6
7,2.8 $\pm$ 0.3,2.78,0.29,2145,1279,4.3 $\pm$ 0.4,4.29,0.37,3.3 $\pm$ 0.3,3.26,...,Tbar_TuneZ2_t-channel,4.4 $\pm$ 0.4,4.45,0.37,nan,4.4 $\pm$ 0.4,4.3 $\pm$ 0.4,3.3 $\pm$ 0.3,2.8 $\pm$ 0.3,2.8 $\pm$ 0.3


In [298]:
df_full = pd.concat([df_mc, df_data])

/usr/local/bin/start_ipykernel.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [353]:
sample_names = df_full["sample"]
df_out = df_full[["tau_requirement", "lep_veto", "met", "btag"]].transpose()

In [354]:
df_out

,0,1,2,3,4,5,6,7,8,9,10,11
tau_requirement,1.0 $\pm$ 0.2,280.6 $\pm$ 18.0,158.9 $\pm$ 6.1,18.0 $\pm$ 0.7,8.2 $\pm$ 0.5,0.4 $\pm$ 0.1,18.8 $\pm$ 0.7,4.4 $\pm$ 0.4,13948,13948,10035,10035
lep_veto,1.0 $\pm$ 0.2,256.3 $\pm$ 17.2,122.3 $\pm$ 5.3,16.1 $\pm$ 0.7,8.1 $\pm$ 0.5,0.4 $\pm$ 0.1,17.1 $\pm$ 0.7,4.3 $\pm$ 0.4,13741,13741,9892,9892
met,0.8 $\pm$ 0.2,237.1 $\pm$ 16.6,96.7 $\pm$ 4.7,13.8 $\pm$ 0.6,5.6 $\pm$ 0.4,0.3 $\pm$ 0.1,14.7 $\pm$ 0.6,3.3 $\pm$ 0.3,7794,7794,6353,6353
btag,0.7 $\pm$ 0.2,42.7 $\pm$ 6.9,19.2 $\pm$ 2.1,10.5 $\pm$ 0.6,4.9 $\pm$ 0.4,0.2 $\pm$ 0.1,11.4 $\pm$ 0.6,2.8 $\pm$ 0.3,1797,5997,1441,4912


In [355]:
sample_names.name = "Cut"

In [356]:
df_out.columns = sample_names

In [357]:
list(sample_names)

['T_TuneZ2_s',
 'WJetsToLNu',
 'DYJetsToLL',
 'T_TuneZ2_tW',
 'T_TuneZ2_t-channel',
 'Tbar_TuneZ2_s',
 'Tbar_TuneZ2_tW',
 'Tbar_TuneZ2_t-channel',
 'Run2011A_MultiJet',
 'Run2011A_MultiJet_QCD',
 'Run2011B_MultiJet',
 'Run2011B_MultiJet_QCD']

In [358]:
df_out

Cut,T_TuneZ2_s,WJetsToLNu,DYJetsToLL,T_TuneZ2_tW,T_TuneZ2_t-channel,Tbar_TuneZ2_s,Tbar_TuneZ2_tW,Tbar_TuneZ2_t-channel,Run2011A_MultiJet,Run2011A_MultiJet_QCD,Run2011B_MultiJet,Run2011B_MultiJet_QCD
tau_requirement,1.0 $\pm$ 0.2,280.6 $\pm$ 18.0,158.9 $\pm$ 6.1,18.0 $\pm$ 0.7,8.2 $\pm$ 0.5,0.4 $\pm$ 0.1,18.8 $\pm$ 0.7,4.4 $\pm$ 0.4,13948,13948,10035,10035
lep_veto,1.0 $\pm$ 0.2,256.3 $\pm$ 17.2,122.3 $\pm$ 5.3,16.1 $\pm$ 0.7,8.1 $\pm$ 0.5,0.4 $\pm$ 0.1,17.1 $\pm$ 0.7,4.3 $\pm$ 0.4,13741,13741,9892,9892
met,0.8 $\pm$ 0.2,237.1 $\pm$ 16.6,96.7 $\pm$ 4.7,13.8 $\pm$ 0.6,5.6 $\pm$ 0.4,0.3 $\pm$ 0.1,14.7 $\pm$ 0.6,3.3 $\pm$ 0.3,7794,7794,6353,6353
btag,0.7 $\pm$ 0.2,42.7 $\pm$ 6.9,19.2 $\pm$ 2.1,10.5 $\pm$ 0.6,4.9 $\pm$ 0.4,0.2 $\pm$ 0.1,11.4 $\pm$ 0.6,2.8 $\pm$ 0.3,1797,5997,1441,4912


In [359]:
row = {"tau_requirement":r"4 jets $+\tau_{h}$", "lep_veto":r"$\mathrm{e}, \mu$ veto" , 
       "met":r"$E_{\mathrm{T}}^{\mathrm{miss}}$", "btag":r"$\geq 1 \mathrm{btag}$"}

#out_df = out_df.rename(index=row)

#header1 = ["Run2011A", "Run2011B", r"$\mathrm{t} \overline{\mathrm{t}}\rightarrow \tau_{h}+$jets", 
#           r"$\mathrm{t} \overline{\mathrm{t}}\rightarrow \mathrm{X}$", "W+jets", "Z+jets", "6", "7"]


In [360]:
df_out = df_out.rename(row)

In [361]:
cols1 = [
 'Run2011A_MultiJet',
 'Run2011B_MultiJet',
 'WJetsToLNu',
 'DYJetsToLL',
]
header1 = [
 'Run2011A',
 'Run2011B',
 r"$\mathrm{t} \overline{\mathrm{t}}\rightarrow \tau_{h}+$jets",
 r"$\mathrm{t} \overline{\mathrm{t}}\rightarrow \mathrm{X}$",
]

In [362]:
cols2 = [
 'T_TuneZ2_s',
 'T_TuneZ2_tW',
 'T_TuneZ2_t-channel',
 'Tbar_TuneZ2_s',
 'Tbar_TuneZ2_tW',
 'Tbar_TuneZ2_t-channel',
]
header2 = [
 'T_TuneZ2_s',
 'T_TuneZ2_tW',
 'T_TuneZ2_t-channel',
 'Tbar_TuneZ2_s',
 'Tbar_TuneZ2_tW',
 'Tbar_TuneZ2_t-channel',
]

In [363]:
print(df_out[cols1].to_latex(escape=False, header=header1))

\begin{tabular}{lllll}
\toprule
Cut & Run2011A & Run2011B & $\mathrm{t} \overline{\mathrm{t}}\rightarrow \tau_{h}+$jets & $\mathrm{t} \overline{\mathrm{t}}\rightarrow \mathrm{X}$ \\
\midrule
4 jets $+\tau_{h}$               &    13948 &    10035 &                                   280.6 $\pm$ 18.0 &                                    158.9 $\pm$ 6.1 \\
$\mathrm{e}, \mu$ veto           &    13741 &     9892 &                                   256.3 $\pm$ 17.2 &                                    122.3 $\pm$ 5.3 \\
$E_{\mathrm{T}}^{\mathrm{miss}}$ &     7794 &     6353 &                                   237.1 $\pm$ 16.6 &                                     96.7 $\pm$ 4.7 \\
$\geq 1 \mathrm{btag}$           &     1797 &     1441 &                                     42.7 $\pm$ 6.9 &                                     19.2 $\pm$ 2.1 \\
\bottomrule
\end{tabular}



In [308]:
df_out[["T_TuneZ2_s", "WJetsToLNu"]]

sample,T_TuneZ2_s,WJetsToLNu
tau_requirement,1.0 $\pm$ 0.2,280.6 $\pm$ 18.0
lep_veto,1.0 $\pm$ 0.2,256.3 $\pm$ 17.2
met,0.8 $\pm$ 0.2,237.1 $\pm$ 16.6
btag,0.7 $\pm$ 0.2,42.7 $\pm$ 6.9
sample,T_TuneZ2_s,WJetsToLNu


In [266]:
sample_names = df["sample"]

In [267]:
df = df.drop(["sample"], axis=1)

In [268]:
for c in df.columns:
    df[c] = df[c].map('{:.1f}'.format)

In [269]:
def format_yield(col, col_err):
    return col + " $\pm$ " + col_err

In [272]:
out_cols = []
for cut in ["tau_requirement_w", "lep_veto_w", "met_w", "btag_w"]:
    
    df[cut + "_out"] = df.apply(lambda row : format_yield(row[cut], row[cut + "_err"]), axis=1)  

In [273]:
mc = df[]

,btag,btag_w,btag_w_err,hlt,jet_requirement,lep_veto,lep_veto_w,lep_veto_w_err,met,met_w,met_w_err,preselected,tau_requirement,tau_requirement_w,tau_requirement_w_err,trigger,tau_requirement_w_out,lep_veto_w_out,met_w_out,btag_w_out
0,22.0,0.7,0.2,273.0,184.0,30.0,1.0,0.2,23.0,0.8,0.2,1915.0,30.0,1.0,0.2,nan,1.0 $\pm$ 0.2,1.0 $\pm$ 0.2,0.8 $\pm$ 0.2,0.7 $\pm$ 0.2
1,40.0,42.7,6.9,4982.0,3163.0,231.0,256.3,17.2,213.0,237.1,16.6,44906.0,252.0,280.6,18.0,nan,280.6 $\pm$ 18.0,256.3 $\pm$ 17.2,237.1 $\pm$ 16.6,42.7 $\pm$ 6.9
2,88.0,19.2,2.1,19961.0,13383.0,547.0,122.3,5.3,431.0,96.7,4.7,114807.0,710.0,158.9,6.1,nan,158.9 $\pm$ 6.1,122.3 $\pm$ 5.3,96.7 $\pm$ 4.7,19.2 $\pm$ 2.1
3,377.0,10.5,0.6,5529.0,3764.0,568.0,16.1,0.7,481.0,13.8,0.6,20110.0,632.0,18.0,0.7,nan,18.0 $\pm$ 0.7,16.1 $\pm$ 0.7,13.8 $\pm$ 0.6,10.5 $\pm$ 0.6
4,147.0,4.9,0.4,3396.0,2007.0,240.0,8.1,0.5,167.0,5.6,0.4,23527.0,245.0,8.2,0.5,nan,8.2 $\pm$ 0.5,8.1 $\pm$ 0.5,5.6 $\pm$ 0.4,4.9 $\pm$ 0.4
5,7.0,0.2,0.1,158.0,101.0,12.0,0.4,0.1,9.0,0.3,0.1,1117.0,13.0,0.4,0.1,nan,0.4 $\pm$ 0.1,0.4 $\pm$ 0.1,0.3 $\pm$ 0.1,0.2 $\pm$ 0.1
6,446.0,11.4,0.6,6034.0,4142.0,648.0,17.1,0.7,556.0,14.7,0.6,21966.0,713.0,18.8,0.7,nan,18.8 $\pm$ 0.7,17.1 $\pm$ 0.7,14.7 $\pm$ 0.6,11.4 $\pm$ 0.6
7,94.0,2.8,0.3,2145.0,1279.0,144.0,4.3,0.4,108.0,3.3,0.3,15056.0,150.0,4.4,0.4,nan,4.4 $\pm$ 0.4,4.3 $\pm$ 0.4,3.3 $\pm$ 0.3,2.8 $\pm$ 0.3
8,1797.0,nan,nan,301139.0,143691.0,13741.0,nan,nan,7794.0,nan,nan,340427.0,13948.0,nan,nan,301139.0,nan $\pm$ nan,nan $\pm$ nan,nan $\pm$ nan,nan $\pm$ nan
9,5997.0,nan,nan,301139.0,143691.0,13741.0,nan,nan,7794.0,nan,nan,340427.0,13948.0,nan,nan,301139.0,nan $\pm$ nan,nan $\pm$ nan,nan $\pm$ nan,nan $\pm$ nan


In [231]:
df = df.transpose()

In [242]:
df.columns = df.loc["sample"]

In [243]:
df

sample,T_TuneZ2_s,WJetsToLNu,DYJetsToLL,T_TuneZ2_tW,T_TuneZ2_t-channel,Tbar_TuneZ2_s,Tbar_TuneZ2_tW,Tbar_TuneZ2_t-channel,Run2011A_MultiJet,Run2011A_MultiJet_QCD,Run2011B_MultiJet,Run2011B_MultiJet_QCD
btag,22,40,88,377,147,7,446,94,1797,5997,1441,4912
btag_w,0.73,42.72,19.15,10.51,4.88,0.19,11.40,2.78,NaN,NaN,NaN,NaN
btag_w_err,0.16,6.86,2.08,0.55,0.42,0.07,0.55,0.29,NaN,NaN,NaN,NaN
hlt,273,4982,19961,5529,3396,158,6034,2145,301139,301139,218730,218730
jet_requirement,184,3163,13383,3764,2007,101,4142,1279,143691,143691,99953,99953
lep_veto,30,231,547,568,240,12,648,144,13741,13741,9892,9892
lep_veto_w,1.02,256.31,122.26,16.12,8.06,0.37,17.09,4.29,NaN,NaN,NaN,NaN
lep_veto_w_err,0.19,17.18,5.33,0.69,0.53,0.11,0.68,0.37,NaN,NaN,NaN,NaN
met,23,213,431,481,167,9,556,108,7794,7794,6353,6353
met_w,0.79,237.10,96.67,13.78,5.65,0.28,14.74,3.26,NaN,NaN,NaN,NaN


In [93]:
df = df.transpose()

In [99]:
df.columns = df.iloc[6]

In [244]:
df.loc[["lep_veto", "btag"]]

sample,T_TuneZ2_s,WJetsToLNu,DYJetsToLL,T_TuneZ2_tW,T_TuneZ2_t-channel,Tbar_TuneZ2_s,Tbar_TuneZ2_tW,Tbar_TuneZ2_t-channel,Run2011A_MultiJet,Run2011A_MultiJet_QCD,Run2011B_MultiJet,Run2011B_MultiJet_QCD
lep_veto,30,231,547,568,240,12,648,144,13741,13741,9892,9892
btag,22,40,88,377,147,7,446,94,1797,5997,1441,4912


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [ ]:
np.hist()

In [81]:
print(df.to_latex(index=False))

\begin{tabular}{rrrrrrlrr}
\toprule
 btag &     hlt &  jet\_requirement &  lep\_veto &   met &  preselected &                 sample &  tau\_requirement &   trigger \\
\midrule
   22 &     273 &              184 &        30 &    23 &         1915 &             T\_TuneZ2\_s &               30 &       NaN \\
   40 &    4982 &             3163 &       231 &   213 &        44906 &             WJetsToLNu &              252 &       NaN \\
   88 &   19961 &            13383 &       547 &   431 &       114807 &             DYJetsToLL &              710 &       NaN \\
  377 &    5529 &             3764 &       568 &   481 &        20110 &            T\_TuneZ2\_tW &              632 &       NaN \\
  147 &    3396 &             2007 &       240 &   167 &        23527 &     T\_TuneZ2\_t-channel &              245 &       NaN \\
    7 &     158 &              101 &        12 &     9 &         1117 &          Tbar\_TuneZ2\_s &               13 &       NaN \\
  446 &    6034 &             4142 &     

In [150]:
from collections import OrderedDict

od = OrderedDict()
for sample, hist in cut_flow_normed.items():
    
    w = []
    w_err = []
    for ibin in range(1, len(hist)-1):
        w.append(hist.GetBinContent(ibin))
        w_err.append(hist.GetBinError(ibin))    
    od[sample] = w
    od[sample + "_err"] = w_err

In [151]:
df = pd.DataFrame(od)

In [152]:
df.rename(index={0: "x", 1: "y", 2: "z", 3:"z"})

,T_TuneZ2_s,T_TuneZ2_s_err,WJetsToLNu,WJetsToLNu_err,DYJetsToLL,DYJetsToLL_err,T_TuneZ2_tW,T_TuneZ2_tW_err,T_TuneZ2_t-channel,T_TuneZ2_t-channel_err,...,Tbar_TuneZ2_t-channel,Tbar_TuneZ2_t-channel_err,Run2011A_MultiJet,Run2011A_MultiJet_err,Run2011A_MultiJet_QCD,Run2011A_MultiJet_QCD_err,Run2011B_MultiJet,Run2011B_MultiJet_err,Run2011B_MultiJet_QCD,Run2011B_MultiJet_QCD_err
x,1.02,0.19,280.63,18.00,158.85,6.08,17.96,0.73,8.24,0.54,...,4.45,0.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
y,1.02,0.19,256.31,17.18,122.26,5.33,16.12,0.69,8.06,0.53,...,4.29,0.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
z,0.79,0.17,237.10,16.55,96.67,4.75,13.78,0.64,5.65,0.45,...,3.26,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
z,0.73,0.16,42.72,6.86,19.15,2.08,10.51,0.55,4.88,0.42,...,2.78,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [153]:
pd.options.display.float_format = '{:,.2f}'.format

In [154]:
print(df.to_latex(index=False))

\begin{tabular}{rrrrrrrrrrrrrrrrrrrrrrrr}
\toprule
 T\_TuneZ2\_s &  T\_TuneZ2\_s\_err &  WJetsToLNu &  WJetsToLNu\_err &  DYJetsToLL &  DYJetsToLL\_err &  T\_TuneZ2\_tW &  T\_TuneZ2\_tW\_err &  T\_TuneZ2\_t-channel &  T\_TuneZ2\_t-channel\_err &  Tbar\_TuneZ2\_s &  Tbar\_TuneZ2\_s\_err &  Tbar\_TuneZ2\_tW &  Tbar\_TuneZ2\_tW\_err &  Tbar\_TuneZ2\_t-channel &  Tbar\_TuneZ2\_t-channel\_err &  Run2011A\_MultiJet &  Run2011A\_MultiJet\_err &  Run2011A\_MultiJet\_QCD &  Run2011A\_MultiJet\_QCD\_err &  Run2011B\_MultiJet &  Run2011B\_MultiJet\_err &  Run2011B\_MultiJet\_QCD &  Run2011B\_MultiJet\_QCD\_err \\
\midrule
       1.02 &            0.19 &      280.63 &           18.00 &      158.85 &            6.08 &        17.96 &             0.73 &                8.24 &                    0.54 &           0.39 &               0.11 &           18.79 &                0.72 &                   4.45 &                       0.37 &               0.00 &                   0.00 &                   0.00 & 

In [155]:
df.columns

Index(['T_TuneZ2_s', 'T_TuneZ2_s_err', 'WJetsToLNu', 'WJetsToLNu_err',
       'DYJetsToLL', 'DYJetsToLL_err', 'T_TuneZ2_tW', 'T_TuneZ2_tW_err',
       'T_TuneZ2_t-channel', 'T_TuneZ2_t-channel_err', 'Tbar_TuneZ2_s',
       'Tbar_TuneZ2_s_err', 'Tbar_TuneZ2_tW', 'Tbar_TuneZ2_tW_err',
       'Tbar_TuneZ2_t-channel', 'Tbar_TuneZ2_t-channel_err',
       'Run2011A_MultiJet', 'Run2011A_MultiJet_err', 'Run2011A_MultiJet_QCD',
       'Run2011A_MultiJet_QCD_err', 'Run2011B_MultiJet',
       'Run2011B_MultiJet_err', 'Run2011B_MultiJet_QCD',
       'Run2011B_MultiJet_QCD_err'],
      dtype='object')

In [156]:
for c in df.columns:
    df[c] = df[c].map('{:.1f}'.format)

In [157]:
def format_yield(col, col_err):
    return col + " $\pm$ " + col_err

In [158]:
df.apply(lambda row : format_yield(row["T_TuneZ2_s"], row["T_TuneZ2_s_err"]), axis=1)

0    1.0 $\pm$ 0.2
1    1.0 $\pm$ 0.2
2    0.8 $\pm$ 0.2
3    0.7 $\pm$ 0.2
dtype: object

In [162]:
out_cols = []
for sample in mc:
    
    out_cols.append(sample + "_out")
    df[sample + "_out"] = df.apply(lambda row : format_yield(row[sample], row[sample + "_err"]), axis=1)  

In [183]:
out_df = df[out_cols]

In [166]:
print(out_df)

  T_TuneZ2_s_out    WJetsToLNu_out   DYJetsToLL_out T_TuneZ2_tW_out  \
0  1.0 $\pm$ 0.2  280.6 $\pm$ 18.0  158.9 $\pm$ 6.1  18.0 $\pm$ 0.7   
1  1.0 $\pm$ 0.2  256.3 $\pm$ 17.2  122.3 $\pm$ 5.3  16.1 $\pm$ 0.7   
2  0.8 $\pm$ 0.2  237.1 $\pm$ 16.6   96.7 $\pm$ 4.7  13.8 $\pm$ 0.6   
3  0.7 $\pm$ 0.2    42.7 $\pm$ 6.9   19.2 $\pm$ 2.1  10.5 $\pm$ 0.6   

  T_TuneZ2_t-channel_out Tbar_TuneZ2_s_out Tbar_TuneZ2_tW_out  \
0          8.2 $\pm$ 0.5     0.4 $\pm$ 0.1     18.8 $\pm$ 0.7   
1          8.1 $\pm$ 0.5     0.4 $\pm$ 0.1     17.1 $\pm$ 0.7   
2          5.6 $\pm$ 0.4     0.3 $\pm$ 0.1     14.7 $\pm$ 0.6   
3          4.9 $\pm$ 0.4     0.2 $\pm$ 0.1     11.4 $\pm$ 0.6   

  Tbar_TuneZ2_t-channel_out  
0             4.4 $\pm$ 0.4  
1             4.3 $\pm$ 0.4  
2             3.3 $\pm$ 0.3  
3             2.8 $\pm$ 0.3  


In [206]:
row = {0:r"4 jets $+\tau_{h}$", 1:r"$\mathrm{e}, \mu$ veto" , 3:r"$\geq 1 \mathrm{btag}$", 
       2:r"$E_{\mathrm{T}}^{\mathrm{miss}}$"}

out_df = out_df.rename(index=row)

header1 = ["Run2011A", "Run2011B", r"$\mathrm{t} \overline{\mathrm{t}}\rightarrow \tau_{h}+$jets", 
           r"$\mathrm{t} \overline{\mathrm{t}}\rightarrow \mathrm{X}$", "W+jets", "Z+jets", "6", "7"]
print(out_df.to_latex(escape=False, header=header1))

\begin{tabular}{lllllllll}
\toprule
{} &       Run2011A &          Run2011B & $\mathrm{t} \overline{\mathrm{t}}\rightarrow \tau_{h}+$jets & $\mathrm{t} \overline{\mathrm{t}}\rightarrow \mathrm{X}$ &         W+jets &         Z+jets &               6 &              7 \\
\midrule
4 jets $+\tau_{h}$               &  1.0 $\pm$ 0.2 &  280.6 $\pm$ 18.0 &                                    158.9 $\pm$ 6.1 &                                     18.0 $\pm$ 0.7 &  8.2 $\pm$ 0.5 &  0.4 $\pm$ 0.1 &  18.8 $\pm$ 0.7 &  4.4 $\pm$ 0.4 \\
$\mathrm{e}, \mu$ veto           &  1.0 $\pm$ 0.2 &  256.3 $\pm$ 17.2 &                                    122.3 $\pm$ 5.3 &                                     16.1 $\pm$ 0.7 &  8.1 $\pm$ 0.5 &  0.4 $\pm$ 0.1 &  17.1 $\pm$ 0.7 &  4.3 $\pm$ 0.4 \\
$E_{\mathrm{T}}^{\mathrm{miss}}$ &  0.8 $\pm$ 0.2 &  237.1 $\pm$ 16.6 &                                     96.7 $\pm$ 4.7 &                                     13.8 $\pm$ 0.6 &  5.6 $\pm$ 0.4 &  0.3 $\pm$ 0.1 &  14.7 $\pm$ 0.

In [207]:
 \rightarrow \tau_{h}+$ jets

SyntaxError: unexpected character after line continuation character (<ipython-input-207-fe55d69ff857>, line 1)

In [189]:
row = {0:r"4 jets $+\tau_{h}$", 1:r"$\mathrm{e}, \mu$ veto" , 3:r"$\geq 1 \mathrm{btag}$", 
       2:r"$E_{\mathrm{T}}^{\mathrm{miss}}$"}

out_df = out_df.rename(index=row)

In [182]:
out_df

,T_TuneZ2_s_out,WJetsToLNu_out,DYJetsToLL_out,T_TuneZ2_tW_out,T_TuneZ2_t-channel_out,Tbar_TuneZ2_s_out,Tbar_TuneZ2_tW_out,Tbar_TuneZ2_t-channel_out
4 jets $+\tau_{h}$,1.0 $\pm$ 0.2,280.6 $\pm$ 18.0,158.9 $\pm$ 6.1,18.0 $\pm$ 0.7,8.2 $\pm$ 0.5,0.4 $\pm$ 0.1,18.8 $\pm$ 0.7,4.4 $\pm$ 0.4
"$\mathrm{e}, \mu$ veto",1.0 $\pm$ 0.2,256.3 $\pm$ 17.2,122.3 $\pm$ 5.3,16.1 $\pm$ 0.7,8.1 $\pm$ 0.5,0.4 $\pm$ 0.1,17.1 $\pm$ 0.7,4.3 $\pm$ 0.4
$E_{\mathrm{T}}^{\mathrm{miss}}>20 \mathrm{GeV},0.8 $\pm$ 0.2,237.1 $\pm$ 16.6,96.7 $\pm$ 4.7,13.8 $\pm$ 0.6,5.6 $\pm$ 0.4,0.3 $\pm$ 0.1,14.7 $\pm$ 0.6,3.3 $\pm$ 0.3
$\geq 1 \mathrm{btag}$,0.7 $\pm$ 0.2,42.7 $\pm$ 6.9,19.2 $\pm$ 2.1,10.5 $\pm$ 0.6,4.9 $\pm$ 0.4,0.2 $\pm$ 0.1,11.4 $\pm$ 0.6,2.8 $\pm$ 0.3


In [130]:
out = df.astype(str) 

In [131]:
out

0    1.02
1    1.02
2    0.79
3    0.73
Name: T_TuneZ2_s, dtype: object

In [116]:
def format_yield(col, col_err):
    
    

,T_TuneZ2_s,T_TuneZ2_s_err,WJetsToLNu,WJetsToLNu_err,DYJetsToLL,DYJetsToLL_err,T_TuneZ2_tW,T_TuneZ2_tW_err,T_TuneZ2_t-channel,T_TuneZ2_t-channel_err,...,Tbar_TuneZ2_t-channel,Tbar_TuneZ2_t-channel_err,Run2011A_MultiJet,Run2011A_MultiJet_err,Run2011A_MultiJet_QCD,Run2011A_MultiJet_QCD_err,Run2011B_MultiJet,Run2011B_MultiJet_err,Run2011B_MultiJet_QCD,Run2011B_MultiJet_QCD_err
0,1.016788,0.187301,280.631165,18.000707,158.851624,6.081340,17.963804,0.727601,8.244175,0.541184,...,4.449273,0.372599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.016788,0.187301,256.306091,17.184883,122.261223,5.329765,16.121239,0.689121,8.060331,0.534770,...,4.285058,0.366373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.785015,0.165203,237.103027,16.553632,96.674721,4.747578,13.779258,0.639633,5.646024,0.449705,...,3.259686,0.321215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.733771,0.157924,42.718502,6.862771,19.150480,2.081694,10.509355,0.552353,4.881046,0.415363,...,2.778148,0.293455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
cut_flow_normed

{'T_TuneZ2_s': <ROOT.TH1F object ("T_TuneZ2_s") at 0x73355f0>,
 'WJetsToLNu': <ROOT.TH1F object ("WJetsToLNu") at 0xa872ac0>,
 'DYJetsToLL': <ROOT.TH1F object ("DYJetsToLL") at 0x7335f20>,
 'T_TuneZ2_tW': <ROOT.TH1F object ("T_TuneZ2_tW") at 0x10b47c10>,
 'T_TuneZ2_t-channel': <ROOT.TH1F object ("T_TuneZ2_t-channel") at 0xa214120>,
 'Tbar_TuneZ2_s': <ROOT.TH1F object ("Tbar_TuneZ2_s") at 0x1881f7b0>,
 'Tbar_TuneZ2_tW': <ROOT.TH1F object ("Tbar_TuneZ2_tW") at 0x6923d70>,
 'Tbar_TuneZ2_t-channel': <ROOT.TH1F object ("Tbar_TuneZ2_t-channel") at 0xee2a9a0>,
 'Run2011A_MultiJet': <ROOT.TH1F object ("Run2011A_MultiJet") at 0x7dea2f0>,
 'Run2011A_MultiJet_QCD': <ROOT.TH1F object ("Run2011A_MultiJet") at 0x904e510>,
 'Run2011B_MultiJet': <ROOT.TH1F object ("Run2011B_MultiJet") at 0x263374b0>,
 'Run2011B_MultiJet_QCD': <ROOT.TH1F object ("Run2011B_MultiJet") at 0x9308070>}

In [67]:
def get_event_yields(h = cut_flow_normed["WJetsToLNu"]):
    for b in range(1, len(h)-1):
        print(h.GetBinContent(b))
        print(h.GetBinError(b))

In [69]:
get_event_yields()

280.63116455078125
18.000707348820107
256.30609130859375
17.1848832467984
237.10302734375
16.55363197018651
42.718502044677734
6.862771125653615


In [77]:
r"\hline & Run2011A & $\mathrm{t} \overline{\mathrm{t}} \rightarrow \tau_{h}+$ jets & $\mathrm{t} \overline{\mathrm{t}} \rightarrow \mathrm{ll}$ & $\mathrm{t} \overline{\mathrm{t}} \rightarrow$ jets & $\mathrm{t} \overline{\mathrm{t}} \rightarrow 1+$ jets  \\
\hline \hline 4 jets $+\tau_{h}$ & 29056 & $577.2 \pm 9.9$ & $2.2 \pm 0.4$ & $156.3 \pm 4.9$ & $77.9 \pm 3.8$  \\
$\mathrm{e}, \mu$ veto & 28951 & $576.7 \pm 9.9$ & $0.6 \pm 0.2$ & $156.3 \pm 4.9$ & $57.1 \pm 3.3$  \\
$\geq 1 \mathrm{btag}$ & 5047 & $459.0 \pm 8.1$ & $0.6 \pm 0.2$ & $118.8 \pm 3.9$ & $45.3 \pm 2.7$ \\
$E_{\mathrm{T}}^{\mathrm{miss}}>20 \mathrm{GeV}$ & 3050 & $409.2 \pm 7.5$ & $0.6 \pm 0.2$ & $65.3 \pm 2.9$ & $41.1 \pm 2.6$ \\
\hline & Data & $\mathrm{t} \overline{\mathrm{t}} \rightarrow \tau_{h}+$ jets & $\mathrm{t} \overline{\mathrm{t}} \rightarrow \mathrm{ll}$ & $\mathrm{t} \overline{\mathrm{t}} \rightarrow$ jets & $\mathrm{t} \overline{\mathrm{t}} \rightarrow 1+$ jets  \\
\hline \hline 4 jets $+\tau_{h}$ & 29056 & $577.2 \pm 9.9$ & $2.2 \pm 0.4$ & $156.3 \pm 4.9$ & $77.9 \pm 3.8$  \\
$\mathrm{e}, \mu$ veto & 28951 & $576.7 \pm 9.9$ & $0.6 \pm 0.2$ & $156.3 \pm 4.9$ & $57.1 \pm 3.3$  \\
$\geq 1 \mathrm{btag}$ & 5047 & $459.0 \pm 8.1$ & $0.6 \pm 0.2$ & $118.8 \pm 3.9$ & $45.3 \pm 2.7$ \\
$E_{\mathrm{T}}^{\mathrm{miss}}>20 \mathrm{GeV}$ & 3050 & $409.2 \pm 7.5$ & $0.6 \pm 0.2$ & $65.3 \pm 2.9$ & $41.1 \pm 2.6$ \\
\hline"

SyntaxError: EOL while scanning string literal (<ipython-input-77-f4d161645747>, line 1)

In [208]:
sample = samples["T_TuneZ2_tW"]
counts = np.ones(len(sample))
weights = sample['norm'] * sample['trigger_weight'] * sample['btag_weight1']

In [209]:
counts.shape

(377,)

In [210]:
import ROOT
from root_numpy import fill_hist

In [211]:
h = ROOT.TH1F("test", "test", 1, 0, 2)

Warning in <TROOT::Append>: Replacing existing TH1: test (Potential memory leak).


In [212]:
fill_hist(h, counts, weights = weights)

In [213]:
for b in range(len(h)):
    print(h.GetBinContent(b))
    print(h.GetBinError(b))
    

0.0
0.0
10.509354591369629
0.5523525943386911
0.0
0.0


In [223]:
np.histogram(counts, bins=1, weights=weights)[0][0]

10.509353685052446

In [224]:
np.sqrt(np.histogram(counts, bins=1, weights=weights**2)[0])[0]

0.5523525943386911

In [214]:
weights.sum()

10.509353685052439

In [242]:
samples.keys()

dict_keys(['T_TuneZ2_s', 'WJetsToLNu', 'DYJetsToLL', 'T_TuneZ2_tW', 'T_TuneZ2_t-channel', 'Tbar_TuneZ2_s', 'Tbar_TuneZ2_tW', 'Tbar_TuneZ2_t-channel', 'TTJets', 'Run2011A_MultiJet', 'Run2011A_MultiJet_QCD', 'Run2011B_MultiJet', 'Run2011B_MultiJet_QCD'])

In [245]:
def rearrange_samples(samples):
    
    # Concat and split MC in signal and background
    new_samples = {}
    new_samples["TTJets_signal"] = samples["TTJets"][samples["TTJets"]["isSignal"] == True]
    new_samples["TTJets_bkg"] = samples["TTJets"][samples["TTJets"]["isBkg"] == True]
    new_samples["WZJets"] = pd.concat([samples['WJetsToLNu'], samples['DYJetsToLL']], axis=0)
    new_samples["STJets"] = pd.concat([samples['T_TuneZ2_s'], samples['T_TuneZ2_tW'], samples['T_TuneZ2_t-channel'], 
                              samples['Tbar_TuneZ2_s'], samples['Tbar_TuneZ2_tW'], 
                              samples['Tbar_TuneZ2_t-channel']], axis=0)

    # Concat the data
    new_samples["Data"] = pd.concat([samples["Run2011A_MultiJet"], samples["Run2011B_MultiJet"]], axis=0)
    new_samples["QCD"] = pd.concat([samples["Run2011A_MultiJet_QCD"], samples["Run2011B_MultiJet_QCD"]], axis=0)
    
    return new_samples

In [246]:
new_samples = rearrange_samples(samples)

In [248]:
new_samples.keys()

dict_keys(['TTJets_signal', 'TTJets_bkg', 'WZJets', 'STJets', 'Data', 'QCD'])

In [ ]:
importlib.reload(btag)
importlib.reload(weights)

def candidates(sample, df, invert_btag = False, njets=-1):
    
    if "Run2011" in sample: isData = True
    else: isData = False

    print( "Processing:", sample,"isData:", isData, "invert_btag:", invert_btag)

    df['nJets'] = df["Jet_pt"].str.len()
    
    # b-tagging
    df["Jet_nbtags"] = df["Jet_csvDisc"].apply( lambda x : btag.count_btags(x, njets=njets) )
    if invert_btag:
        df = btag.no_tag(df)
    else:
        df = btag.at_least_1tag(df)

    # MET cut
    df = selection.met_requirement(df)

    
    # HL features
    #df = pd.concat([df, df.apply(lambda ev : pd.Series(hl.hlFeatures(ev, njets=njets)), axis=1)], axis=1)
    

    # MC weights
    if not isData:

        hlt_40, hlt_45 = weights.lumi()
        total_lumi = hlt_40 + hlt_45
        trigger_frac = hlt_40 / float(hlt_45)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(btag.eval_sf_eff(ev)), axis=1)], axis=1)
        df["Jet_btag_weight1"] = df.apply(lambda ev : btag.b_weight_method1(ev, njets=njets), axis=1)
        #df["Jet_btag_weight2"] = df.apply(lambda ev : btag.b_weight_method2(ev, njets=njets), axis=1)
        # trigger weights
        #df["trigger_weight"] = df.apply(lambda ev : weights.trigger_weight(ev, trigger_frac), axis=1)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(weights.trigger_weight(ev, trigger_frac)), axis=1)], axis=1)
        # normalization
        counts_path = "/eos/user/l/llayer/opendata_files/preselection_merged/" + sample + "_counts.root"
        total_counts = root_pandas.read_root(counts_path)
        xsec = weights.get_xsec(sample)
        weights.norm(df, total_counts, xsec, lumi = total_lumi)

    # QCD
    if isData & invert_btag:
        
        # Assume light flavour
        def lf(nJets):
            return np.zeros((nJets))
        df["Jet_flavour"] = df["nJets"].apply(lf)
        df = pd.concat([df, df.apply(lambda ev: pd.Series(btag.eval_sf_eff(ev)), axis=1)], axis=1)
        df["btag_weight"] = df.apply(lambda ev : btag.b_weight_method2(ev, njets=njets), axis=1)
        
    return df

In [ ]:
def rearrange_samples(samples):
    
    # Concat and split MC in signal and background
    new_samples = {}
    new_samples["TTJets_signal"], new_samples["TTJets_bkg"] = weights.classify_tt(samples["TTJets"])
    new_samples["WZJets"] = pd.concat([samples['WJetsToLNu'], samples['DYJetsToLL']], axis=0)
    new_samples["STJets"] = pd.concat([samples['T_TuneZ2_s'], samples['T_TuneZ2_tW'], samples['T_TuneZ2_t-channel'], 
                              samples['Tbar_TuneZ2_s'], samples['Tbar_TuneZ2_tW'], 
                              samples['Tbar_TuneZ2_t-channel']], axis=0)

    # Concat the data
    new_samples["Data"] = pd.concat([samples["Run2011A_MultiJet"], samples["Run2011B_MultiJet"]], axis=0)
    new_samples["QCD"] = pd.concat([samples["QCD_Run2011A_MultiJet"], samples["QCD_Run2011B_MultiJet"]], axis=0)
    
    return new_samples

In [ ]:
def proc_candidates(samples, njets = -1):
    
    cand_samples = {}
    for sample in data:
        cand_samples[sample] = candidates(sample, samples[sample], invert_btag = False, njets=njets)
        cand_samples["QCD_" + sample] = candidates(sample, samples[sample], invert_btag = True, njets=njets)
    
    
    for sample in mc:
        cand_samples[sample] = candidates(sample, samples[sample], invert_btag = False, njets=njets)   

    new_samples = rearrange_samples(cand_samples)
    return new_samples

In [ ]:
cands = proc_candidates(samples)

In [ ]:
len(cands["Data"])

In [ ]:
import ROOT
from root_numpy import fill_hist

def save_var(sample, name, var_name, bins = 20, xlow = 0., xup = 350):

    hist = ROOT.TH1D(name + "_" + var_name, name + "_" + var_name, bins, xlow, xup)
    hist.Sumw2()
    
    if name == "Data":
        pass
    elif name == "QCD":
        if var_name == "bdt":
            scale_qcd = 9.
            sample = sample[sample["train_flag"] == "test"]
        else:
            scale_qcd = 4.3
        sample['weight'] = sample['btag_weight'] * scale_qcd
    else:
        #samples[sample]['new_trigger_weight'] = new_samples[sample].apply(lambda ev : weights.trigger_weight(ev), axis=1)
        sample['weight'] = sample['norm'] * (1/1000) * sample['trigger_weight'] * sample['Jet_btag_weight1']
        #print(sample, sum(samples[sample]['weight']))
        #new_samples[sample]['btag_weight2']
    
    # Flatten if the column is a list
    if "Jet_" in var_name:
        series = sample[var_name].apply(pd.Series).stack().reset_index(drop=True)
        if name != "Data":
            sample['weight_stacked'] = sample.apply(lambda x : stack_weight(x["weight"] ,x["nJets"]), axis=1)
            weights = sample['weight_stacked'].apply(pd.Series).stack().reset_index(drop=True)
    else:
        series = sample[var_name]
        if name != "Data":
            weights = sample["weight"]
    if name == "Data":
        fill_hist(hist, series)
    else:
        fill_hist(hist, series, weights = weights)
    #hist.Write()
    return hist

def vars_to_histos(samples, variables):
    
    hists = {}
    for name, sample in samples.items():
        for var in variables:
            hists[name + "_" + var["var_name"]] = save_var(sample, name, var["var_name"], var["bins"], var["xlow"], var["xup"])

    return hists

In [ ]:
variables = [
    {"var_name" : "MET_met", "bins" : 30, "xlow" : 0., "xup" : 400, "xtitle" : "MET [GeV]"}
]

hists = vars_to_histos(cands, variables)

In [ ]:
f = ROOT.TFile("histos_test.root", "RECREATE")
for s, hist in hists.items():

    print( s, hist.Integral() )
    hist.Write()
    
f.Close()

In [ ]:
def fit(histos, fit_var, corr = "central"):
        

    bkg = histos['TTJets_bkg' + "_" + fit_var ].Clone()
    bkg.Add(histos['WZJets' + "_" + fit_var ])
    bkg.Add(histos['STJets' + "_" + fit_var ])

    data = histos["Data" + "_" + fit_var ].Clone()
    data.Add(bkg, -1.)

    signal = histos["TTJets_signal" + "_" + fit_var ]

    qcd = histos["QCD" + "_" + fit_var ]

    x = ROOT.RooRealVar("x","x",0.,350.)
    rooSignal =  ROOT.RooDataHist("signal","signal",ROOT.RooArgList( x ), signal)
    rooBkg = ROOT.RooDataHist("bkg","bkg", ROOT.RooArgList( x ), qcd)
    signal_pdf = ROOT.RooHistPdf("signal","signal",ROOT.RooArgSet( x ), rooSignal)
    bkg_pdf = ROOT.RooHistPdf("bkg","bkg",ROOT.RooArgSet( x ), rooBkg)

    c0 = ROOT.RooRealVar("c0","c0",0.5,0.,1.)
    pdf = ROOT.RooAddPdf("pdf","pdf", signal_pdf,bkg_pdf, c0)

    dataFit = ROOT.RooDataHist("data","data",ROOT.RooArgList( x ), data);

    fitResult = pdf.fitTo(dataFit)
    #print( fitResult )

    sf_tt_sig = (c0.getVal() * data.Integral()) / signal.Integral() 
    sf_tt_sig_err = (c0.getError() * data.Integral()) / signal.Integral()
    
    sf_qcd = ((1-c0.getVal())*data.Integral()) / qcd.Integral()
    sf_qcd_err = (c0.getError() * data.Integral()) / qcd.Integral()
    
    
    print( "scale factor TTbar tau(h) QQ ", sf_tt_sig, "+-", sf_tt_sig_err )
    print( "scale factor MultiJet ", sf_qcd, "+-", sf_qcd_err )

    return sf_tt_sig, sf_qcd

In [ ]:
fit(hists, "MET_met")